In [1]:
import import_ipynb
import pickle
import numpy as np
from matplotlib import pyplot as plt
from copy import deepcopy as dc

In [2]:
def objects(grid,ofunc,oarg='all',embed='full',strict=False,):
    def zerogrid(nrows,ncols):
        return [[0 for c in range(ncols)] for l in range(nrows)]
    def bbox(component):
        rmax=max([r for ((r,c),v) in component])
        rmin=min([r for ((r,c),v) in component])
        cmax=max([c for ((r,c),v) in component])
        cmin=min([c for ((r,c),v) in component])
        return (rmax-rmin+1,cmax-cmin+1),(rmin,cmin)
    def update(r,c,v,z,rmin=0,cmin=0):
        z[r-rmin][c-cmin]=v
        return
    def traverse_component(pos,grid,strict=False):
        """Returns no. of unseen elements connected to (i,j)."""
        i, j = pos
        result = 1
        component = [(pos,grid[i][j])]
        # Check all four neighbours
        if strict==False: nlist=[(i-1, j), (i+1, j), (i, j-1), (i, j+1),(i-1, j-1), (i+1, j-1), (i-1, j+1), (i+1, j+1)]
        elif strict==True: nlist=[(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
        for new_pos in nlist:
            if new_pos in elements:
                elements.remove(new_pos)
                component+=traverse_component(pos=new_pos,grid=grid)
        return component
    def connected_components(grid,valuefn,arg=[],strict=False):
        # Tracks size of largest connected component found
        global elements
        elements = set((i, j) for i, line in enumerate(grid) for j, cell in enumerate(line) if valuefn(cell,arg))
        componentL=[]
        while len(elements)>0:
            pos = elements.pop()
            componentL+=[traverse_component(pos,grid,strict=strict)]
        return componentL
    def components_to_objects(componentL):
        objL=[]
        offL=[]
        for component in componentL:
            if embed=='bbox': box,offset=bbox(component)
            elif embed=='full': box,offset=(len(grid),len(grid[0])),(0,0)
            zg=zerogrid(box[0],box[1])
            [update(r,c,v,zg,offset[0],offset[1]) for ((r,c),v) in [c for c in component]]
            objL+=[zg]
            offL+=[offset]
        return objL,offL
    allobjectsL=[]
    alloffsetsL=[]
    if oarg=='all': 
        elementL=list(set([item for sublist in grid for item in sublist]))
        elementL.remove(0)
        for e in elementL:
            componentL=connected_components(grid,ofunc,[e],strict=strict)
            objectsL,offsetsL=components_to_objects(componentL)
            allobjectsL+=objectsL
            alloffsetsL+=offsetsL
    elif type(oarg)==list or oarg=='any':
        if oarg=='any': oargL=ist(set([item for sublist in grid for item in sublist]))
        else: oargL=oarg
        componentL=componentL=connected_components(grid,ofunc,oargL,strict=strict)
        components_to_objects(componentL)
        objectsL,offsetsL=components_to_objects(componentL)
        allobjectsL+=objectsL
        alloffsetsL+=offsetsL
    return allobjectsL,alloffsetsL

In [3]:
def assemble_grid(objectsL,offsetsL,orig_Nr,orig_Nc,embed='full',scale=1):
    zg=0
    def zerogrid(nrows,ncols):
        return [[0 for c in range(ncols)] for l in range(nrows)]
    def place(r,c,v,rmin=0,cmin=0):
        zg[r+rmin][c+cmin]=v
        return
    if embed=='bbox':
        objL=[]
        zg=zerogrid(orig_Nr,orig_Nc)
        for obj,off in zip(objectsL,offsetsL):
            nrows,ncols=len(obj),len(obj[0])
            [place(r,c,obj[r][c],off[0],off[1]) for r in range(nrows) for c in range(ncols)]
        objL+=[zg]
    elif embed=='full': objL=objectsL
    if scale>1 and embed=='bbox':
        scaledL=[]
        Mr=int(orig_Nr*scale)
        Mc=int(orig_Nc*scale)
        delta_r=int((M-orig_Nr)/2)+1
        delta_c=int((M-orig_Nc)/2)+1
        zg=zerogrid(M,M)
        for obj in objL:
            [place(r,c,obj[r][c],delta_r,delta_c) for r in range(orig_Nr) for c in range(orig_Nc) if obj[r][c]!=0]
            scaledL+=[zg]
        objL=scaledL
    aL=[np.array(obj) for obj in objL]
    return np.sum(aL,axis=0).tolist()

In [141]:
noteq=lambda cell,arg: True if cell not in arg else False

In [142]:
member=lambda cell,arg: True if cell in arg else False

In [4]:
class Machine():
    def __init__(self):
        self.input=[]
        self.memory=[]
        self.aux=[]
        self.output=[]
        self.target=[]
        self.debug=False
        self.log={}
        self.counter=0
    def load(self,grid,target):
        self.input=[grid]
        self.target=target
    def copy(self):
        self.memory=self.input
        self.print_state('copy')
    def out(self):
        self.output+=self.memory
        self.print_state('out')
    def apply(self,func,*args,fname=None):
        args=[self.memory]+list(args)
        self.memory=func(*args)
        self.print_state(fname)
    def print_state(self,arg):
        if self.debug:
            logitem=[]
            logitem+=[('FUNCTION:',arg)]
            logitem+=[('input:',dc(self.input))]
            logitem+=[('memory:',dc(self.memory))]
            logitem+=[('output:',dc(self.output))]
            self.log[self.counter]=logitem
            self.counter+=1